# Chapter 6 Enginereing Sustainability: Sustainable Geotech

1. [Introduction: Sustainable Geotech](#1.-Introduction)
2. [Simulation: Sustainable Geotech](#2.-Simulation)
3. [Self-Assessment](#3.-Self-Assessment)

## 1. Introduction

### 🧱 Theoretical Background: Sustainable Geotechnical Engineering Calculator

---

### 📘 Overview

This calculator estimates the environmental and structural performance of common foundation systems—**shallow footings**, **pile foundations**, and **retaining walls**—based on user-defined geometry and soil parameters. It integrates:

- Engineering design models for bearing capacity, axial pile capacity, and earth pressure
- Material quantity estimation for concrete, steel, and geosynthetics
- Embodied carbon and life cycle cost calculations
- A sustainability scoring system based on reuse, emissions, and cost

---

### 📐 Engineering Models

### 1. **Shallow Foundation Bearing Capacity**  
Based on Terzaghi’s equation for strip footings:

$$
q_{\text{ult}} = c N_c + q N_q + 0.5 \gamma B N_\gamma
$$

- $( c $): soil cohesion (kPa)  
- $( q $): overburden pressure (kPa)  
- $( \gamma $): unit weight of soil (kN/m³)  
- $( B $): footing width (m)  
- $( N_c, N_q, N_\gamma $): bearing capacity factors (empirical)

### 2. **Pile Foundation Axial Capacity**  
Simplified model combining skin friction and end bearing:

$$
Q_{\text{ult}} = \alpha c \pi L + c A_p N_c
$$

- $( \alpha $): adhesion factor (typically 0.6)  
- $( c $): cohesion  
- $( L $): pile length (m)  
- $( A_p $): pile cross-sectional area (m²)  
- $( N_c $): bearing capacity factor (typically 9 for cohesive soils)

### 3. **Retaining Wall Earth Pressure**  
Using Rankine’s active earth pressure theory:

$$
P_a = \frac{1}{2} \gamma H^2 K_a
\quad \text{where} \quad
K_a = \frac{1 - \sin(\phi)}{1 + \sin(\phi)}
$$

- $( \gamma $): unit weight of soil  
- $( H $): wall height (m)  
- $( \phi $): internal friction angle (°)

---

### 🌍 Sustainability Metrics

### 1. **Embodied Carbon**  
Calculated using emission factors:

| Material       | Emission Factor (kg CO₂/kg) |
|----------------|-----------------------------|
| Concrete       | 0.15                        |
| Steel          | 1.85                        |
| Geosynthetics  | 2.50                        |

$$
\text{Carbon} = \sum (\text{mass}_i \times \text{emission factor}_i)
$$

### 2. **Life Cycle Cost**  
Estimated using unit costs:

| Material       | Cost ($/kg) |
|----------------|-------------|
| Concrete       | 0.10        |
| Steel          | 1.20        |
| Geosynthetics  | 2.00        |

$$
\text{Cost} = \sum (\text{mass}_i \times \text{unit cost}_i)
$$

### 3. **Sustainability Score**  
Empirical scoring system (0–10):

- +2 if reuse ≥ 50%  
- +2 if carbon < 500 kg CO₂  
- +1 if cost < $1000

---

### 📊 Material Estimation

Concrete volume is calculated from geometry:

- Shallow footing: $( V = B \times B \times D $)  
- Pile: $( V = A_p \times L $)  
- Retaining wall: $( V = H \times t_{\text{wall}} \times L_{\text{wall}} $)

Steel mass is estimated using typical reinforcement ratios:

| Foundation Type     | Steel Ratio (kg/m³) |
|---------------------|---------------------|
| Shallow Footing     | 100                 |
| Pile Foundation     | 120                 |
| Retaining Wall      | 150                 |

---

### 📚 References and Sources

- Terzaghi, K. (1943). *Theoretical Soil Mechanics*. Wiley.
- Das, B.M. (2010). *Principles of Foundation Engineering*. Cengage.
- Rankine, W.J.M. (1857). *On the Stability of Retaining Walls*. Philosophical Transactions.
- Hammond, G.P., & Jones, C.I. (2008). *Embodied energy and carbon in construction materials*. ICE Proceedings.
- World Bank (2020). *Low Carbon Construction Materials: Cost and Emissions Benchmarks*.


### 📥 Typical Input Ranges

| Parameter            | Typical Range       |
|----------------------|---------------------|
| Cohesion $( c $)     | 0–100 kPa           |
| Unit Weight $( \gamma $) | 16–22 kN/m³     |
| Friction Angle $( \phi $) | 25–40°         |
| Reuse Percentage     | 0–100%              |
| Concrete Density     | 2400 kg/m³          |
| Steel Reinforcement  | 100–150 kg/m³       |

---

## 2. Simulation

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Emission and cost factors
emission_factors = {"concrete": 0.15, "steel": 1.85, "geosynthetic": 2.5}
cost_factors = {"concrete": 0.10, "steel": 1.20, "geosynthetic": 2.00}

# Engineering models
def bearing_capacity(c, gamma, B, q):
    Nc, Nq, Ngamma = 5.14, 1.0, 0.0
    return c * Nc + q * Nq + 0.5 * gamma * B * Ngamma

def pile_capacity(c, Ap, L, alpha=0.6):
    Qs = alpha * c * np.pi * L
    Qp = c * Ap * 9
    return Qs + Qp

def earth_pressure(gamma, H, phi):
    Ka = (1 - np.sin(np.radians(phi))) / (1 + np.sin(np.radians(phi)))
    return 0.5 * gamma * H**2 * Ka

def sustainability_score(reuse_pct, carbon, cost):
    score = 5
    if reuse_pct >= 50: score += 2
    if carbon < 500: score += 2
    if cost < 1000: score += 1
    return max(min(score, 10), 0)

# Material estimation
def estimate_materials(foundation_type, B, D, Ap, L, H, t_wall, wall_length):
    if foundation_type == "Shallow":
        V_conc = B * B * D
        steel_ratio = 100
    elif foundation_type == "Pile":
        V_conc = Ap * L
        steel_ratio = 120
    elif foundation_type == "Retaining Wall":
        V_conc = H * t_wall * wall_length
        steel_ratio = 150
    else:
        V_conc = 0
        steel_ratio = 0
    concrete_kg = V_conc * 2400
    steel_kg = V_conc * steel_ratio
    return concrete_kg, steel_kg

# Widgets
foundation_dropdown = widgets.Dropdown(
    options=["Shallow", "Pile", "Retaining Wall", "Geosynthetic-Reinforced"],
    description='Type'
)
reuse_slider = widgets.FloatSlider(value=0, min=0, max=100, step=10, description='Reuse (%)')
geo_slider = widgets.FloatSlider(value=200, min=0, max=1000, step=50, description='Geosynthetics (kg)')

# Geometry
B_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='Footing Width (m)')
D_slider = widgets.FloatSlider(value=0.5, min=0.2, max=2.0, step=0.1, description='Footing Depth (m)')
Ap_slider = widgets.FloatSlider(value=0.2, min=0.05, max=1.0, step=0.05, description='Pile Area (m²)')
L_slider = widgets.FloatSlider(value=10, min=2, max=30, step=1, description='Pile Length (m)')
H_slider = widgets.FloatSlider(value=4, min=1, max=10, step=0.5, description='Wall Height (m)')
t_wall_slider = widgets.FloatSlider(value=0.3, min=0.2, max=1.0, step=0.05, description='Wall Thickness (m)')
wall_length_slider = widgets.FloatSlider(value=6, min=2, max=20, step=1, description='Wall Length (m)')

# Soil
c_slider = widgets.FloatSlider(value=25, min=0, max=100, step=5, description='Cohesion (kPa)')
gamma_slider = widgets.FloatSlider(value=18, min=10, max=22, step=1, description='Unit Weight (kN/m³)')
q_slider = widgets.FloatSlider(value=30, min=0, max=100, step=5, description='Overburden (kPa)')
phi_slider = widgets.FloatSlider(value=30, min=20, max=45, step=1, description='Friction Angle (°)')

# Output and button
output = widgets.Output()
run_button = widgets.Button(description="Run Analysis", button_style='success')

# Update visible inputs
def update_visibility(change=None):
    with output:
        clear_output()
        base = [foundation_dropdown, reuse_slider, geo_slider]
        if foundation_dropdown.value == "Shallow":
            inputs = base + [B_slider, D_slider, c_slider, gamma_slider, q_slider]
        elif foundation_dropdown.value == "Pile":
            inputs = base + [Ap_slider, L_slider, c_slider]
        elif foundation_dropdown.value == "Retaining Wall":
            inputs = base + [H_slider, t_wall_slider, wall_length_slider, gamma_slider, phi_slider]
        else:
            inputs = base
        display(widgets.VBox(inputs + [run_button]))

# Compute results
def compute_results(button=None):
    with output:
        clear_output(wait=True)
        concrete_kg, steel_kg = estimate_materials(
            foundation_dropdown.value,
            B_slider.value, D_slider.value,
            Ap_slider.value, L_slider.value,
            H_slider.value, t_wall_slider.value,
            wall_length_slider.value
        )
        concrete_eff = concrete_kg * (1 - reuse_slider.value / 100)
        steel_eff = steel_kg * (1 - reuse_slider.value / 100)
        geo_eff = geo_slider.value * (1 - reuse_slider.value / 100)

        carbon = (concrete_eff * emission_factors["concrete"] +
                  steel_eff * emission_factors["steel"] +
                  geo_eff * emission_factors["geosynthetic"])
        cost = (concrete_eff * cost_factors["concrete"] +
                steel_eff * cost_factors["steel"] +
                geo_eff * cost_factors["geosynthetic"])
        score = sustainability_score(reuse_slider.value, carbon, cost)

        if foundation_dropdown.value == "Shallow":
            result = bearing_capacity(c_slider.value, gamma_slider.value, B_slider.value, q_slider.value)
            label = "Ultimate Bearing Capacity (kPa)"
        elif foundation_dropdown.value == "Pile":
            result = pile_capacity(c_slider.value, Ap_slider.value, L_slider.value)
            label = "Axial Pile Capacity (kN)"
        elif foundation_dropdown.value == "Retaining Wall":
            result = earth_pressure(gamma_slider.value, H_slider.value, phi_slider.value)
            label = "Active Earth Pressure (kN/m)"
        else:
            result = "N/A"
            label = "Performance Metric"
        # 📘 Interpretation and Summary
        print("\n📘 Interpretation:")

        # Embodied carbon interpretation
        if carbon < 500:
            print("✅ Low embodied carbon — aligns with sustainable construction targets.")
        elif carbon < 800:
            print("⚠️ Moderate embodied carbon — better than conventional but could improve.")
        else:
            print("❌ High embodied carbon — consider reducing concrete or increasing reuse.")

        # Cost interpretation
        if cost < 1000:
            print("✅ Economical design — suitable for budget-sensitive projects.")
        elif cost < 1500:
            print("⚠️ Moderate cost — acceptable for performance-driven applications.")
        else:
            print("❌ High cost — consider optimizing material use or sourcing.")

        # Strength interpretation
        if foundation_dropdown.value == "Shallow" and result < 150:
            print("⚠️ Low bearing capacity — may not be suitable for heavy loads.")
        elif foundation_dropdown.value == "Pile" and result < 500:
            print("⚠️ Low axial capacity — consider increasing pile length or diameter.")
        elif foundation_dropdown.value == "Retaining Wall" and result < 20:
            print("⚠️ Low earth pressure — verify soil parameters.")
        else:
            print("✅ Structural performance appears adequate for typical conditions.")

        # Sustainability score interpretation
        if score >= 8:
            print("✅ High sustainability score — excellent balance of cost, carbon, and reuse.")
        elif score >= 5:
            print("⚠️ Moderate sustainability — room for improvement.")
        else:
            print("❌ Low sustainability — redesign recommended.")

        # 📊 Summary Table
        print("\n📊 Summary Compared to Typical Values:")
        print(f"{'Metric':<25}{'Your Design':>15}{'Typical Range':>20}")
        print("-" * 60)
        print(f"{'Embodied Carbon (kg CO₂)':<25}{carbon:>15.1f}{'300–800':>20}")
        print(f"{'Life Cycle Cost ($)':<25}{cost:>15.2f}{'800–1500':>20}")
        print(f"{label:<25}{result if isinstance(result, str) else f'{result:.1f}':>15}{'Depends on type':>20}")
        print(f"{'Sustainability Score':<25}{score:>15}{'5–10':>20}")

        
        
        print(f"🏗️ Foundation Type: {foundation_dropdown.value}")
        print(f"Concrete Volume: {concrete_kg / 2400:.2f} m³")
        print(f"Concrete Used: {concrete_eff:.1f} kg")
        print(f"Steel Used: {steel_eff:.1f} kg")
        print(f"Geosynthetics Used: {geo_eff:.1f} kg")
        print(f"Reuse Percentage: {reuse_slider.value}%")
        print(f"\n🌍 Embodied Carbon: {carbon:.2f} kg CO₂")
        print(f"💰 Life Cycle Cost: ${cost:.2f}")
        print(f"📏 {label}: {result if isinstance(result, str) else f'{result:.1f}'}")
        print(f"🧠 Sustainability Score: {score}/10")

        categories = ['Carbon (kg CO₂)', 'Cost ($)', 'Sustainability']
        values = [carbon, cost, score]
        colors = ['gray', 'green', 'blue']

        plt.figure(figsize=(7, 4))
        plt.bar(categories, values, color=colors)
        plt.title("Sustainable Geotechnical Metrics")
        plt.ylabel("Value")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Link triggers
foundation_dropdown.observe(update_visibility, names='value')
run_button.on_click(compute_results)

# Initial display
update_visibility()
display(output)

Output()

## 3. Self-Assessment

### 🧱 Reflective, Conceptual, and Quiz Questions: Sustainable Geotechnical Engineering Calculator

---

### 📘 Conceptual Questions

These questions explore the engineering principles, sustainability metrics, and modeling logic behind the calculator.

### Foundation Design
- What are the key differences in load transfer mechanisms between shallow and deep foundations?
- How does geometry (e.g., footing width, pile length, wall height) influence material demand and structural performance?
- Why is bearing capacity estimated using Terzaghi’s equation for shallow foundations?

### Material Estimation
- How is concrete volume estimated from geometric inputs, and why is density used to convert to mass?
- What assumptions are made about steel reinforcement ratios in different foundation types?
- How does material reuse affect both embodied carbon and cost?

### Sustainability Metrics
- Why is embodied carbon a critical metric in geotechnical design?
- How does the sustainability score balance environmental and economic performance?
- What are the limitations of using fixed emission and cost factors across diverse project contexts?

---

### 🔍 Reflective Questions

These questions encourage critical thinking and application to real-world design decisions.

- If your project site has soft clay and high water table, which foundation type would you choose and why?
- How would your design change if the goal was to minimize embodied carbon rather than maximize bearing capacity?
- What trade-offs exist between increasing wall thickness for strength and reducing material use for sustainability?
- How could this model be extended to include excavation impacts or construction equipment emissions?
- What policy or procurement strategies could incentivize reuse of foundation materials?

---

### ❓ Quiz Questions

### Multiple Choice

1. **Which foundation type typically requires the greatest volume of concrete per unit area?**  
   A. Shallow footing  
   B. Pile foundation  
   C. Retaining wall  
   D. Geosynthetic-reinforced soil  
   **Answer:** B

2. **What is the primary factor influencing axial capacity of a pile in cohesive soil?**  
   A. Pile diameter  
   B. Soil friction angle  
   C. Cohesion and pile length  
   D. Wall height  
   **Answer:** C

3. **Which parameter most directly affects active earth pressure on a retaining wall?**  
   A. Wall thickness  
   B. Soil cohesion  
   C. Friction angle and wall height  
   D. Pile area  
   **Answer:** C

### True/False

4. **Increasing reuse percentage reduces both embodied carbon and material cost.**  
   **Answer:** True

5. **The sustainability score increases if the design uses more concrete.**  
   **Answer:** False

6. **Steel reinforcement is assumed to scale linearly with concrete volume in this model.**  
   **Answer:** True

### Short Answer

7. **Explain how the calculator estimates concrete mass from geometric inputs.**  
   *Answer:* It calculates volume based on foundation geometry and multiplies by concrete density (2400 kg/m³) to estimate mass.

8. **What are the environmental benefits of using geosynthetics in foundation systems?**  
   *Answer:* Geosynthetics can reduce the need for bulk materials, improve soil performance, and enable reuse or modular design, lowering overall carbon footprint.

---